In [1]:
from database.market import Market
from database.sec import SEC
from database.adatabase import ADatabase
from modeler.modeler import Modeler as m
from processor.processor import Processor as p
from datetime import datetime, timedelta
import pytz
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pickle

In [2]:
market = Market()
sec = SEC()

In [3]:
market.connect()
sp5 = market.retrieve("sp500")
market.disconnect()

In [4]:
sec.connect()
training_data = []
training_columns = {}
for ticker in tqdm(list(sp5["Symbol"].unique())):
    try:
        cik = sp5[sp5["Symbol"]==ticker]["CIK"].item()
        filing = sec.retrieve_filing_data(cik)
        ticker_financials = filing.rename(columns={"filed":"date"})
        ticker_financials["date"] = [datetime.strptime(str(x),"%Y%m%d") for x in ticker_financials["date"]]
        ticker_financials["year"] = [x.year for x in ticker_financials["date"]]
        ticker_financials["quarter"] = [x.quarter for x in ticker_financials["date"]]
        ticker_financials = p.column_date_processing(ticker_financials)
        for col in ticker_financials.columns:
            if col in training_columns.keys():
                training_columns[col] += 1
            else:
                training_columns[col] = 1
        ticker_financials["ticker"] = ticker
    except Exception as e:
        print(ticker,str(e))
sec.disconnect()

 39%|██████████████████████████████████████████████████████████▉                                                                                            | 197/505 [01:04<02:18,  2.22it/s]

FRC 'date'


 83%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                          | 417/505 [02:15<00:37,  2.33it/s]

SBNY 'date'


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 505/505 [02:42<00:00,  3.11it/s]


In [5]:
unrequired = ['adsh',
 'cik',
 'name',
 'sic',
 'countryba',
 'stprba',
 'cityba',
 'zipba',
 'bas1',
 'bas2',
 'baph',
 'countryma',
 'stprma',
 'cityma',
 'zipma',
 'mas1',
 'mas2',
 'countryinc',
 'stprinc',
 'ein',
 'former',
 'changed',
 'afs',
 'wksi',
 'fye',
 'form',
 'period',
 'fy',
 'fp',
 'date',
 'accepted',
 'prevrpt',
 'detail',
 'instance',
 'nciks',
 'aciks']

In [10]:
factors = [x for x in training_columns if x not in unrequired and training_columns[x] >= 475]

In [11]:
factors

['accumulatedothercomprehensiveincomelossnetoftax',
 'assets',
 'cashandcashequivalentsatcarryingvalue',
 'cashandcashequivalentsperiodincreasedecrease',
 'deferredincometaxexpensebenefit',
 'earningspersharebasic',
 'earningspersharediluted',
 'entitycommonstocksharesoutstanding',
 'entitypublicfloat',
 'incometaxexpensebenefit',
 'liabilitiesandstockholdersequity',
 'netcashprovidedbyusedinfinancingactivities',
 'netcashprovidedbyusedininvestingactivities',
 'netcashprovidedbyusedinoperatingactivities',
 'netincomeloss',
 'propertyplantandequipmentnet',
 'retainedearningsaccumulateddeficit',
 'stockholdersequity',
 'weightedaveragenumberofdilutedsharesoutstanding',
 'weightedaveragenumberofsharesoutstandingbasic',
 'year',
 'quarter',
 'comprehensiveincomenetoftax',
 'operatingleasesfutureminimumpaymentsdueintwoyears',
 'unrecognizedtaxbenefits',
 'cashcashequivalentsrestrictedcashandrestrictedcashequivalentsperiodincreasedecreaseincludingexchangerateeffect',
 'lesseeoperatingleaseli

In [22]:
sec.connect()
training_data = []
training_columns = {}
plausible_pool = []
for ticker in tqdm(list(sp5["Symbol"].unique())):
    try:
        cik = sp5[sp5["Symbol"]==ticker]["CIK"].item()
        filing = sec.retrieve_filing_data(cik)
        ticker_financials = filing.rename(columns={"filed":"date"})
        ticker_financials["date"] = [datetime.strptime(str(x),"%Y%m%d") for x in ticker_financials["date"]]
        ticker_financials["year"] = [x.year for x in ticker_financials["date"]]
        ticker_financials["quarter"] = [x.quarter for x in ticker_financials["date"]]
        ticker_financials = p.column_date_processing(ticker_financials)
        ticker_financials["ticker"] = ticker
        col_count = 0
        for col in factors:
            if col not in ticker_financials.columns:
                continue
            else:
                col_count += 1
        if col_count == len(factors):
            plausible_pool.append(ticker)
    except Exception as e:
        print(str(e))
sec.disconnect()

 39%|██████████████████████████████████████████████████████████▉                                                                                            | 197/505 [01:04<02:19,  2.20it/s]

'date'


 83%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                          | 417/505 [02:15<00:38,  2.26it/s]

'date'


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 505/505 [02:42<00:00,  3.10it/s]


In [21]:
len(plausible_pool)

0

In [27]:
sec.connect()
financial_data = []
for ticker in tqdm(plausible_pool):
    try:
        cik = sp5[sp5["Symbol"]==ticker]["CIK"].item()
        filing = sec.retrieve_filing_data(cik)
        ticker_financials = filing.rename(columns={"filed":"date"})
        ticker_financials["date"] = [datetime.strptime(str(x),"%Y%m%d") for x in ticker_financials["date"]]
        ticker_financials["year"] = [x.year for x in ticker_financials["date"]]
        ticker_financials["quarter"] = [x.quarter for x in ticker_financials["date"]]
        ticker_financials = p.column_date_processing(ticker_financials)
        final_set = ticker_financials[factors]
        final_set["ticker"] = ticker
        financial_data.extend(final_set.to_dict("records"))
    except Exception as e:
        print(str(e))
sec.disconnect()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 304/304 [01:38<00:00,  3.08it/s]


In [28]:
standardized_set = pd.DataFrame(financial_data)

In [31]:
dq = []
for col in standardized_set.columns:
    val = len([x for x in standardized_set[col].isna() if x == True]) / standardized_set.index.size
    dq.append({"col":col,"rate":val})

In [34]:
column_rates = pd.DataFrame(dq).sort_values("rate")

In [39]:
parse_two = standardized_set[column_rates[column_rates["rate"] < 0.10]["col"]]

In [47]:
final_parse = []
for ticker in list(parse_two["ticker"].unique()):
    ticker_data = parse_two[parse_two["ticker"]==ticker]
    for col in ticker_data.columns:
        try:
            ticker_data[col].fillna(ticker_data[col].mean(),inplace=True)
        except:
            continue
    final_parse.extend(ticker_data.to_dict("records"))

In [48]:
final_set = pd.DataFrame(final_parse)

In [50]:
dq = []
for col in final_set.columns:
    val = len([x for x in final_set[col].isna() if x == True]) / final_set.index.size
    dq.append({"col":col,"rate":val})

In [51]:
column_rates = pd.DataFrame(dq).sort_values("rate")
column_rates

,col,rate
0,ticker,0.0
1,quarter,0.0
2,year,0.0
3,assets,0.0
4,liabilitiesandstockholdersequity,0.0
5,incometaxexpensebenefit,0.0
6,retainedearningsaccumulateddeficit,0.0
7,accumulatedothercomprehensiveincomelossnetoftax,0.0
8,earningspersharebasic,0.0
9,earningspersharediluted,0.0


In [54]:
market.connect()
market.store("financial_set",final_set)
market.disconnect()

In [55]:
final_set["year"].max()

2022